In [1]:
from os import chdir
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
chdir("/content/drive/MyDrive/Sherbrooke/")

!pip install tweet-preprocessor
%pip install ipython-autotime
%pip install optuna

Mounted at /content/drive
     |████████████████████████████████| 308 kB 7.7 MB/s 
     |████████████████████████████████| 210 kB 63.7 MB/s 
     |████████████████████████████████| 81 kB 12.0 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 113 kB 71.9 MB/s 
     |████████████████████████████████| 146 kB 68.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=2415a4869b4d577f114dd6f42f9247b6caf43fb8674b2b373b872b99032ebdc0
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
!pip install -r requirements.txt

In [3]:
# IMPORTS

import preprocessing as pp
import hyperparameters as hp
import RNN as rnn
import optimising as opt
import json
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier

%load_ext autoreload
%autoreload 2

%load_ext autotime


time: 695 µs (started: 2022-04-18 15:44:57 +00:00)


In [4]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
# 3 classes
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising=None, original_class=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising=None, original_class=False)
# 5 classes
X_train_5, y_train_5 = pp.prepare_dataframe(file_name_train, True, lemmatising=False)
X_test_5, y_test_5 = pp.prepare_dataframe(file_name_test, True, lemmatising=False)

time: 24.9 s (started: 2022-04-18 15:45:28 +00:00)


In [5]:
pipeline_sgd = hp.pipeline_sgd
pipeline_gb = hp.pipeline_gb
pipeline_rf = hp.pipeline_rf
pipeline_lr = hp.pipeline_lr
pipeline_per = hp.pipeline_per
pipeline_svc = hp.pipeline_svc

time: 2.28 ms (started: 2022-04-18 15:46:13 +00:00)


# Hyperparameters Tuning

### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [8]:
with open('data.json') as json_file:
    dico = json.load(json_file)

time: 313 ms (started: 2022-04-18 15:46:40 +00:00)


### SGD

In [ ]:
%%time
grid_sgd = hp.grid_search("SGD", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Score:  0.8668915077832064
Best Params:  {'clf__alpha': 1e-05, 'clf__penalty': 'l1', 'tfidf__max_df': 0.3, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 14.4 s, sys: 2.6 s, total: 17 s
Wall time: 8min 30s
time: 8min 30s (started: 2022-04-15 16:22:47 +00:00)


In [ ]:
hp.if_save("SGD", dico, grid_sgd)

#### No new best score for  SGD
time: 2.05 ms (started: 2022-04-15 16:31:18 +00:00)


### Gradient boosting

In [ ]:
%%time
grid_gb = hp.grid_search("GB", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Score:  0.581
Best Params:  {'clf__learning_rate': 0.2, 'clf__n_estimators': 200, 'tfidf__max_df': 0.3, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}
CPU times: user 7.96 s, sys: 490 ms, total: 8.45 s
Wall time: 9min 54s
time: 9min 54s (started: 2022-04-15 16:31:18 +00:00)


In [ ]:
hp.if_save("GB", dico, grid_gb)

#### No new best score for  GB
time: 1.63 ms (started: 2022-04-15 16:41:12 +00:00)


### Random Forest

In [ ]:
%%time
grid_rf = hp.grid_search("RF", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Score:  0.594
Best Params:  {'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'tfidf__max_df': 0.6, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}
CPU times: user 8.42 s, sys: 369 ms, total: 8.79 s
Wall time: 2min 35s
time: 2min 35s (started: 2022-04-15 18:26:41 +00:00)


In [ ]:
hp.if_save('RF', dico, grid_rf)

#### No new best score for  RF
time: 1.59 ms (started: 2022-04-15 18:29:16 +00:00)


### Logistic Regression

In [ ]:
%%time
grid_lr = hp.grid_search("LR", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best Score:  0.6140000000000001
Best Params:  {'clf__C': 10, 'clf__tol': 1e-09, 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 15.7 s, sys: 2.83 s, total: 18.5 s
Wall time: 4min 41s
time: 4min 41s (started: 2022-04-15 22:24:12 +00:00)


In [ ]:
hp.if_save('LR', dico, grid_lr)

#### No new best score for  LR
time: 2.95 ms (started: 2022-04-15 22:28:54 +00:00)


### Perceptron

In [ ]:
%%time
grid_per = hp.grid_search("PER", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best Score:  0.8097043608680832
Best Params:  {'clf__alpha': 5.05e-06, 'clf__penalty': 'l1', 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 1min 3s, sys: 8.39 s, total: 1min 11s
Wall time: 37min 26s
time: 37min 26s (started: 2022-04-15 22:45:14 +00:00)


In [ ]:
hp.if_save('PER', dico, grid_per)

#### No new best score for  PER
time: 2.77 ms (started: 2022-04-15 23:58:39 +00:00)


### LinearSVC

In [ ]:
%%time
grid_svc = hp.grid_search("SVC", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
720 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-package

Best Score:  0.8602693724198008
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1', 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 1min 1s, sys: 6.82 s, total: 1min 8s
Wall time: 44min 48s
time: 44min 48s (started: 2022-04-15 23:58:54 +00:00)


In [ ]:
hp.if_save('SVC', dico, grid_svc)

#### No new best score for  SVC
time: 2.54 ms (started: 2022-04-16 00:43:43 +00:00)


### Charging the best parameters for training

In [13]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)

data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['PER']
data_svc = data['SVC']

time: 261 ms (started: 2022-04-18 15:50:04 +00:00)


### Training on all training data and testing on test data

In [21]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  \n")
    print('------------------------------------------------------\n')
    print('\tClassification Report Train : \n\n', classification_report(y_train, predictions_train, target_names=['Negative','Neutral' ,'Positive']))
    print('------------------------------------------------------\n')
    print('\tClassification Report Test:\n\n',classification_report(y_test,predictions_test, target_names=['Negative', 'Neutral', 'Positive']))
    print('------------------------------------------------------\n')


time: 6.7 ms (started: 2022-04-18 15:52:38 +00:00)


In [27]:
def testing_5_classes(model):
    predictions_train = model.predict(X_train_5)
    predictions_test = model.predict(X_test_5)
    accuracy_train = accuracy_score(y_train_5,predictions_train )
    accuracy_test = accuracy_score(y_test_5,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  \n")
    print('------------------------------------------------------\n')
    print('\tClassification Report Train : \n\n', classification_report(y_train_5, predictions_train, target_names=['Extremely Negative', 'Negative','Neutral' ,'Positive', 'Extremely Positive']))
    print('------------------------------------------------------\n')
    print('\tClassification Report Test:\n\n',classification_report(y_test_5,predictions_test, target_names=['Extremely Negative', 'Negative','Neutral' ,'Positive', 'Extremely Positive']))
    print('------------------------------------------------------\n')


time: 5.1 ms (started: 2022-04-18 15:55:04 +00:00)


### SGD

In [16]:
data_sgd

{'best_score': 0.8769590395125022,
 'params': {'clf__alpha': 1.2000000000000002e-05,
  'clf__penalty': 'l1',
  'tfidf__max_df': 0.75,
  'tfidf__min_df': 1,
  'tfidf__ngram_range': [1, 1]}}

time: 3.52 ms (started: 2022-04-18 15:51:35 +00:00)


In [17]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**(data_sgd["params"]),clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 8.52 ms (started: 2022-04-18 15:51:41 +00:00)


**3 Classes**

In [19]:
%%time

model_sgd_best.fit(X_train,y_train)


CPU times: user 1.96 s, sys: 106 ms, total: 2.06 s
Wall time: 1.64 s


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 1.65 s (started: 2022-04-18 15:52:16 +00:00)


In [22]:
print("3 CLASSES")
testing(model_sgd_best)

3 CLASSES
train_accuracy : 0.9316855813495483 
test_accuracy : 0.8700370566437269  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.93      0.93      0.93     15317
     Neutral       0.92      0.89      0.90      7392
    Positive       0.94      0.95      0.94     17912

    accuracy                           0.93     40621
   macro avg       0.93      0.92      0.93     40621
weighted avg       0.93      0.93      0.93     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.88      0.87      0.88      1631
     Neutral       0.83      0.83      0.83       604
    Positive       0.87      0.88      0.88      1543

    accuracy                           0.87      3778
   macro avg       0.86      0.86      0.86      3778
weighted avg       0.87      0.

**5 CLASSES**

In [23]:
%%time

model_sgd_best.fit(X_train_5,y_train_5)

CPU times: user 3.05 s, sys: 213 ms, total: 3.26 s
Wall time: 2.26 s


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 2.28 s (started: 2022-04-18 15:53:21 +00:00)


In [28]:
print("5 CLASSES")
testing_5_classes(model_sgd_best)

5 CLASSES
train_accuracy : 0.8041407153935156 
test_accuracy : 0.5976707252514558  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.80      0.86      0.83      5470
          Negative       0.81      0.73      0.77      9847
           Neutral       0.81      0.90      0.85      7392
          Positive       0.81      0.74      0.78     11300
Extremely Positive       0.78      0.88      0.83      6612

          accuracy                           0.80     40621
         macro avg       0.80      0.82      0.81     40621
      weighted avg       0.80      0.80      0.80     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.62      0.65      0.64       592
          Negative       0.57      0.47      0.51      1039
           Neu

### Random Forest

In [29]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**(data_rf["params"]),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 7.41 ms (started: 2022-04-18 15:55:12 +00:00)


In [30]:
%%time

model_rf_best.fit(X_train,y_train)

CPU times: user 22.6 s, sys: 67.7 ms, total: 22.7 s
Wall time: 12.1 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 12.1 s (started: 2022-04-18 15:55:15 +00:00)


In [32]:
print("3 CLASSES")
testing(model_rf_best)

3 Classes
train_accuracy : 0.8655867654661382 
test_accuracy : 0.6971942826892535  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.90      0.89      0.90     15317
     Neutral       0.90      0.59      0.71      7392
    Positive       0.83      0.96      0.89     17912

    accuracy                           0.87     40621
   macro avg       0.88      0.81      0.83     40621
weighted avg       0.87      0.87      0.86     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.76      0.67      0.71      1631
     Neutral       0.74      0.40      0.52       604
    Positive       0.64      0.84      0.73      1543

    accuracy                           0.70      3778
   macro avg       0.71      0.64      0.65      3778
weighted avg       0.71      0.

In [33]:
%%time

model_rf_best.fit(X_train_5,y_train_5)

CPU times: user 23.8 s, sys: 73.6 ms, total: 23.8 s
Wall time: 14.4 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 14.4 s (started: 2022-04-18 15:56:29 +00:00)


In [35]:
print("5 CLASSES")
testing_5_classes(model_rf_best)

5 CLASSES
train_accuracy : 0.8283154033627926 
test_accuracy : 0.4563260984647962  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.97      0.68      0.80      5470
          Negative       0.83      0.87      0.85      9847
           Neutral       0.82      0.81      0.82      7392
          Positive       0.74      0.93      0.82     11300
Extremely Positive       0.96      0.73      0.83      6612

          accuracy                           0.83     40621
         macro avg       0.87      0.81      0.83     40621
      weighted avg       0.85      0.83      0.83     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.67      0.23      0.34       592
          Negative       0.45      0.44      0.45      1039
           Neu

### Gradient Boosting 

In [36]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**(data_gb["params"]))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=200))])

time: 6.54 ms (started: 2022-04-18 15:57:06 +00:00)


**3 Classes**

In [37]:
%%time
model_gb_best.fit(X_train,y_train)

CPU times: user 3min 45s, sys: 730 ms, total: 3min 46s
Wall time: 3min 54s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=200))])

time: 3min 54s (started: 2022-04-18 15:57:11 +00:00)


In [38]:
print("3 CLASSES")
testing(model_gb_best)

train_accuracy : 0.7982816769651165 
test_accuracy : 0.7511911064055056  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.85      0.79      0.82     15317
     Neutral       0.75      0.65      0.70      7392
    Positive       0.78      0.87      0.82     17912

    accuracy                           0.80     40621
   macro avg       0.79      0.77      0.78     40621
weighted avg       0.80      0.80      0.80     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.84      0.72      0.77      1631
     Neutral       0.67      0.62      0.64       604
    Positive       0.71      0.84      0.77      1543

    accuracy                           0.75      3778
   macro avg       0.74      0.72      0.73      3778
weighted avg       0.76      0.75      0.

**5 Classes**

In [39]:
%%time
model_gb_best.fit(X_train_5,y_train_5)

CPU times: user 5min 48s, sys: 948 ms, total: 5min 49s
Wall time: 5min 50s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=200))])

time: 5min 50s (started: 2022-04-18 16:01:07 +00:00)


In [40]:
print("5 CLASSES")
testing_5_classes(model_gb_best)

5 CLASSES
train_accuracy : 0.685212082420423 
test_accuracy : 0.5473795659078877  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.80      0.68      0.73      5470
          Negative       0.70      0.60      0.65      9847
           Neutral       0.70      0.73      0.72      7392
          Positive       0.58      0.74      0.65     11300
Extremely Positive       0.83      0.67      0.74      6612

          accuracy                           0.69     40621
         macro avg       0.72      0.68      0.70     40621
      weighted avg       0.70      0.69      0.69     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.63      0.48      0.55       592
          Negative       0.54      0.45      0.49      1039
           Neut

### Logistic Regression

In [41]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**(data_lr['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 6.22 ms (started: 2022-04-18 16:07:00 +00:00)


**3 Classes**

In [42]:
%%time

model_lr_best.fit(X_train,y_train)

CPU times: user 1.08 s, sys: 72.1 ms, total: 1.15 s
Wall time: 21.6 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 21.6 s (started: 2022-04-18 16:07:00 +00:00)


In [43]:
print('3 CLASSES')
testing(model_lr_best)

3 CLASSES
train_accuracy : 0.9775239408187884 
test_accuracy : 0.8213340391741663  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.97      0.98      0.98     15317
     Neutral       0.99      0.94      0.97      7392
    Positive       0.98      0.99      0.98     17912

    accuracy                           0.98     40621
   macro avg       0.98      0.97      0.98     40621
weighted avg       0.98      0.98      0.98     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.85      0.82      0.83      1631
     Neutral       0.72      0.72      0.72       604
    Positive       0.83      0.86      0.85      1543

    accuracy                           0.82      3778
   macro avg       0.80      0.80      0.80      3778
weighted avg       0.82      0.

**5 Classes**

In [44]:
%%time

model_lr_best.fit(X_train_5,y_train_5)

CPU times: user 1.19 s, sys: 48.4 ms, total: 1.24 s
Wall time: 53.7 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 53.8 s (started: 2022-04-18 16:07:23 +00:00)


In [45]:
print("5 CLASSES")
testing_5_classes(model_lr_best)

5 CLASSES
train_accuracy : 0.9548755569779178 
test_accuracy : 0.5907887771307571  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.96      0.96      0.96      5470
          Negative       0.95      0.95      0.95      9847
           Neutral       0.98      0.95      0.96      7392
          Positive       0.94      0.96      0.95     11300
Extremely Positive       0.96      0.96      0.96      6612

          accuracy                           0.95     40621
         macro avg       0.96      0.96      0.96     40621
      weighted avg       0.96      0.95      0.95     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.63      0.52      0.57       592
          Negative       0.54      0.55      0.55      1039
           Neu

### Perceptron

In [46]:
model_per_best = pipeline_per
model_per_best.set_params(**(data_per['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 6.48 ms (started: 2022-04-18 16:08:18 +00:00)


**3 CLASSES**

In [47]:
%%time
model_per_best.fit(X_train,y_train)

CPU times: user 1.52 s, sys: 133 ms, total: 1.66 s
Wall time: 1.45 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 1.46 s (started: 2022-04-18 16:08:18 +00:00)


In [48]:
print("3 CLASSES")
testing(model_per_best)

3 CLASSES
train_accuracy : 0.904089017995618 
test_accuracy : 0.8194812069878242  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.91      0.91      0.91     15317
     Neutral       0.88      0.80      0.84      7392
    Positive       0.91      0.94      0.92     17912

    accuracy                           0.90     40621
   macro avg       0.90      0.88      0.89     40621
weighted avg       0.90      0.90      0.90     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.85      0.82      0.84      1631
     Neutral       0.72      0.69      0.70       604
    Positive       0.82      0.87      0.84      1543

    accuracy                           0.82      3778
   macro avg       0.80      0.79      0.80      3778
weighted avg       0.82      0.8

**5 Classes**

In [49]:
%%time
model_per_best.fit(X_train_5,y_train_5)

CPU times: user 2.11 s, sys: 168 ms, total: 2.28 s
Wall time: 1.69 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 1.7 s (started: 2022-04-18 16:08:21 +00:00)


In [50]:
print("5 CLASSES")
testing_5_classes(model_per_best)

5 CLASSES
train_accuracy : 0.7280470692498954 
test_accuracy : 0.5034409740603494  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.86      0.68      0.76      5470
          Negative       0.62      0.80      0.70      9847
           Neutral       0.82      0.69      0.75      7392
          Positive       0.72      0.65      0.68     11300
Extremely Positive       0.77      0.84      0.81      6612

          accuracy                           0.73     40621
         macro avg       0.76      0.73      0.74     40621
      weighted avg       0.74      0.73      0.73     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.62      0.42      0.50       592
          Negative       0.43      0.55      0.48      1039
           Neu

### SVC

In [51]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**(data_svc['params']))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 5.4 ms (started: 2022-04-18 16:08:26 +00:00)


**3 Classes**



In [52]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 6.66 s, sys: 29 ms, total: 6.69 s
Wall time: 6.67 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 6.68 s (started: 2022-04-18 16:08:27 +00:00)


In [53]:
print("3 CLASSES")
testing(model_svc_best)

3 CLASSES
train_accuracy : 0.9460623815267964 
test_accuracy : 0.8642138697723664  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.95      0.95     15317
     Neutral       0.95      0.90      0.93      7392
    Positive       0.95      0.96      0.95     17912

    accuracy                           0.95     40621
   macro avg       0.95      0.94      0.94     40621
weighted avg       0.95      0.95      0.95     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.88      0.86      0.87      1631
     Neutral       0.81      0.80      0.80       604
    Positive       0.87      0.89      0.88      1543

    accuracy                           0.86      3778
   macro avg       0.85      0.85      0.85      3778
weighted avg       0.86      0.

**5 Classes**

In [54]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 7.3 s, sys: 30 ms, total: 7.33 s
Wall time: 7.66 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 7.67 s (started: 2022-04-18 16:08:35 +00:00)


In [55]:
print("5 CLASSES")
testing_5_classes(model_svc_best)

5 CLASSES
train_accuracy : 0.6524457792767288 
test_accuracy : 0.5603493912122817  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.00      0.00      0.00      5470
          Negative       0.59      0.93      0.72      9847
           Neutral       0.95      0.90      0.93      7392
          Positive       0.59      0.94      0.72     11300
Extremely Positive       0.00      0.00      0.00      6612

          accuracy                           0.65     40621
         macro avg       0.43      0.56      0.47     40621
      weighted avg       0.48      0.65      0.55     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.00      0.00      0.00       592
          Negative       0.52      0.80      0.63      1039
           Neu

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

### Voting Classifier

In [56]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


time: 4.25 ms (started: 2022-04-18 16:08:44 +00:00)


**Voting Classifier 1**

**3 Classes**

In [57]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 3min 45s, sys: 1.02 s, total: 3min 46s
Wall time: 4min 22s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.75,
                                                               ngram_range=[1,
                                                                            1])),
                                              ('clf',
                                               SGDClassifier(alpha=1.2000000000000002e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(max_depth=600,
                                                                      max_features='sqrt',
                               

time: 4min 22s (started: 2022-04-18 16:08:53 +00:00)


In [58]:
print("3 CLASSES")
testing(eclf1)

3 CLASSES
train_accuracy : 0.9528322788705349 
test_accuracy : 0.860772895712017  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.97      0.95     15317
     Neutral       0.97      0.90      0.93      7392
    Positive       0.95      0.96      0.96     17912

    accuracy                           0.95     40621
   macro avg       0.96      0.94      0.95     40621
weighted avg       0.95      0.95      0.95     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.87      0.87      0.87      1631
     Neutral       0.83      0.78      0.81       604
    Positive       0.86      0.88      0.87      1543

    accuracy                           0.86      3778
   macro avg       0.85      0.85      0.85      3778
weighted avg       0.86      0.8

**5 Classes**

In [59]:
%%time

eclf1.fit(X_train_5,y_train_5)

CPU times: user 6min 14s, sys: 1.59 s, total: 6min 15s
Wall time: 7min 21s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.75,
                                                               ngram_range=[1,
                                                                            1])),
                                              ('clf',
                                               SGDClassifier(alpha=1.2000000000000002e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(max_depth=600,
                                                                      max_features='sqrt',
                               

time: 7min 21s (started: 2022-04-18 16:13:24 +00:00)


In [60]:
print("5 CLASSES")
testing_5_classes(eclf1)

5 CLASSES
train_accuracy : 0.8993623987592625 
test_accuracy : 0.6069348861831657  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.90      0.91      0.90      5470
          Negative       0.88      0.90      0.89      9847
           Neutral       0.92      0.91      0.92      7392
          Positive       0.89      0.90      0.89     11300
Extremely Positive       0.93      0.88      0.90      6612

          accuracy                           0.90     40621
         macro avg       0.90      0.90      0.90     40621
      weighted avg       0.90      0.90      0.90     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.62      0.59      0.61       592
          Negative       0.55      0.54      0.55      1039
           Neu

**Voting Classifier 2**

**3 Classes**

In [61]:
%%time

eclf2.fit(X_train,y_train)

CPU times: user 13 s, sys: 311 ms, total: 13.3 s
Wall time: 39.1 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.75,
                                                               ngram_range=[1,
                                                                            1])),
                                              ('clf',
                                               SGDClassifier(alpha=1.2000000000000002e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                      

time: 39.1 s (started: 2022-04-18 16:20:55 +00:00)


In [62]:
print("3 CLASSES")
testing(eclf2)

3 CLASSES
train_accuracy : 0.9553432953398489 
test_accuracy : 0.8647432503970355  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.97      0.96     15317
     Neutral       0.96      0.92      0.94      7392
    Positive       0.96      0.96      0.96     17912

    accuracy                           0.96     40621
   macro avg       0.96      0.95      0.95     40621
weighted avg       0.96      0.96      0.96     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.87      0.88      0.87      1631
     Neutral       0.81      0.79      0.80       604
    Positive       0.88      0.88      0.88      1543

    accuracy                           0.86      3778
   macro avg       0.85      0.85      0.85      3778
weighted avg       0.86      0.

In [63]:
%%time

eclf2.fit(X_train_5,y_train_5)

CPU times: user 19.9 s, sys: 456 ms, total: 20.3 s
Wall time: 1min 9s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.75,
                                                               ngram_range=[1,
                                                                            1])),
                                              ('clf',
                                               SGDClassifier(alpha=1.2000000000000002e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                      

time: 1min 9s (started: 2022-04-18 16:21:40 +00:00)


In [64]:
print("5 CLASSES")
testing_5_classes(eclf2)

5 CLASSES
train_accuracy : 0.88210531498486 
test_accuracy : 0.5989941768131286  

------------------------------------------------------

	Classification Report Train : 

                     precision    recall  f1-score   support

Extremely Negative       0.86      0.92      0.89      5470
          Negative       0.86      0.87      0.87      9847
           Neutral       0.91      0.92      0.91      7392
          Positive       0.89      0.85      0.87     11300
Extremely Positive       0.90      0.88      0.89      6612

          accuracy                           0.88     40621
         macro avg       0.88      0.89      0.89     40621
      weighted avg       0.88      0.88      0.88     40621

------------------------------------------------------

	Classification Report Test:

                     precision    recall  f1-score   support

Extremely Negative       0.60      0.63      0.62       592
          Negative       0.54      0.53      0.54      1039
           Neutr